## Imports

In [1]:
%pip install transformers
%pip install sklearn
%pip install pytorch

from transformers import BertModel, BertTokenizer,TrainingArguments, Trainer, BatchEncoding

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import pandas as pd
from sklearn.model_selection import train_test_split
import random
import numpy as np
import re

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-uwdzr73g/pytorch_735a2009ccac469d957fb4b97750b849/setup.py", line 15, in <module>
          raise Exception(message)
      Exception: You tried to install "pytorch". The package named for PyTorch is "torch"
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
ERROR: Could not build wheels for pytorch, which is required to install pyproject.toml-based projects


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


ModuleNotFoundError: No module named 'torch'

## Preparação do dataset

In [ ]:
class SequenceViabilityDataset:
  def __init__(self, sequences, labels):
    
    self.sequences = sequences
    self.labels = labels

  def __getitem__(self, i):
    return self.sequences[i], self.labels[i]

  def __len__(self):
    return len(self.sequences)

  def get_idx_split(self, train_size=0.8):
    all_ids = range(len(self.sequences))
    train_ids, test_ids = train_test_split(all_ids, train_size = 0.8)
    test_ids, val_ids = train_test_split(test_ids, train_size = 0.5)
    return {"train": train_ids, "test": test_ids, "valid": val_ids}

## Load dos dados

In [ ]:
df = pd.read_csv("./datasets/protein_mutation_viability.csv")

## Preprocessing dos dados

In [ ]:
all_data = []
for index, row in df.iterrows():
  spl = row.array[0].split(";")
  s, l = spl[0], spl[-1]
  l = 0 if l == "FALSE" else 1
  s = " ".join(list(s.upper()))
  s = re.sub(r"[UZOB]", "X", s)
  all_data.append((s, l))

## Instanciação do dataset

In [ ]:
dataset = SequenceViabilityDataset([x[0] for x in all_data], [x[1] for x in all_data])

## Train test validation split

In [ ]:
split_idx = dataset.get_idx_split()
train_data = [dataset[i] for i in split_idx["train"]]
test_data = [dataset[i] for i in split_idx["test"]]
valid_data = [dataset[i] for i in split_idx["valid"]]

## Criação de batches

In [ ]:

BATCH_SIZE = 10

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)
valid_loader = DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False)

c = 0
for i, l in train_loader:
  if c < 5:
    print(i, l)
  c += 1

('D E D E I A C T N P V A T E Q Y G S V A T N L E R G N E K D', 'D E E E I R T T N P V A T E Q Y G S V H T N L Q R F N R', 'C E E E I A C T N P V A T E S Y G V C T A C A A N N E S E T E H G N D T Q', 'D E E E I R T T N P V A T E Q Y G S V S T A L D Q R S G N E R N', 'D E E E I R T T N P V A T E Q Y G S V S T N L Q C R F G N R', 'D E E E I R T T N P V A T E Q Y G S V S T L N T G G Q G G N T', 'Q V P E G I I L Q T T H C V I V H A T K Q Y G I V E T Y N L Q R W A N R', 'F E T G E I R T T S N P D A A T E Q P T G N K Q S S M N L A C C G Y N Y R P', 'M E D E V C T T N G P V A T E Q Y W G S V V S I T M N L Q M H E N N R L', 'D E E E I S T T N P V A Y E Q Y G Q C V C S W G E H N A E N F N S E T Q A') tensor([1, 1, 1, 1, 1, 1, 0, 0, 0, 1])
('D E D E I A T T N P V A T E Q W G C V A G T G P P N S M N P Q D H Q', 'D E E E I R T T N P V A T E Q Y G S V E E V A T G L Q G R G V V E L F', 'D E G E I R T T N F V A T E Q Y G S V D E E S N L E Q R T G N M', 'S M E L I M T C N P T A S D C F A F L H A N Y I

## Definição do classifier

In [ ]:
class ProtBERTClassifier(torch.nn.Module):
    def __init__(self, model, num_classes):
        super(ProtBERTClassifier, self).__init__()
        self.model = model
        self.classify = torch.nn.Linear(model.config.hidden_size, num_classes)

    def forward(self, sequences):

        bert_embeddings = self.model(**BatchEncoding(data=sequences)).last_hidden_state.to(device)

        return self.classify(bert_embeddings).squeeze()

## Instanciação do classifier

In [ ]:
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False)

bert_model = BertModel.from_pretrained("Rostlab/prot_bert")
bert_model.to(device)
model = ProtBERTClassifier(bert_model, 1)
model.to(device)

Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ProtBERTClassifier(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30, 1024, padding_idx=0)
      (position_embeddings): Embedding(40000, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-29): 30 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-1

## Definição do training loop

In [ ]:
def test_model(model, tokenizer, test_dataloader):
    num_correct = 0
    num_tests = 0
    with torch.no_grad():
        for batched_sequences, labels in test_dataloader:
            #print(batched_sequences, labels)
            tokens = tokenizer(batched_sequences, return_tensors='pt', padding=True).to(device)
            #print(2)
            pred = model(tokens)
            num_correct += (pred.argmax(1) == torch.Tensor(labels).to(device)).sum().item()
            num_tests += len(labels)

    accuracy = num_correct / num_tests
    print("acc =", accuracy)
    return accuracy

def train_model(model, tokenizer, train_dataloader, valid_dataloader, epochs=10, lr=0.01):
  
  optimizer = torch.optim.Adam(model.parameters(), lr)

  patience = 3
  delta = 0.5
  c = 0
  min_so_far = np.inf

  for epoch in range(epochs):
      print("Epoch", epoch+1)

      # Training loss
      for batched_sequences, labels in train_dataloader:


          tokens = tokenizer(batched_sequences, return_tensors='pt', padding=True).to(device)

          pred = model(tokens)

          training_loss = F.cross_entropy(pred, torch.Tensor(labels).to(device))

          optimizer.zero_grad()

          training_loss.backward()

          optimizer.step()
      # Early stopping
      # Comparar validation loss com training loss
      
      test_model(model, tokenizer, test_loader)

      # Sem afetar o modelo
      total_validation_loss = 0
      with torch.no_grad():
        for batched_sequences, labels in valid_dataloader:

            tokens = tokenizer(batched_sequences, return_tensors='pt', padding=True).to(device)

            pred = model(tokens)

            valid_loss = F.cross_entropy(pred, torch.Tensor(labels).to(device))
            
            total_validation_loss += valid_loss.item()
            

      # Se a validation loss for a menor até agora, atualizar e dar reset ao counter
      if total_validation_loss < min_so_far:
        min_so_far = valid_loss.item()
        c = 0
      # Se a validation loss for maior que a menor até agora + delta, incrementar counter
      elif total_validation_loss > min_so_far + delta:
        c += 1

        # Se a validation_loss aumentou vezes suficientes de seguida, parar
        if c >= patience:
          break
  return model

In [ ]:

trained_model = train_model(model, tokenizer, train_loader, valid_loader, epochs=25)

TypeError: train_model() got multiple values for argument 'epochs'